# ANÁLISIS EXPLORATORIO DE DATOS

## 1. Importar Librerias

In [1]:
# Librerias

import pandas as pd
import os
from pathlib import Path

## 2. Traer Datos

### 2.1. Traer Datos de Defunciones de 1979 - 1991

In [ ]:
# Ruta de los archivos
ruta_muertes = Path("data/raw/Muertes")

# Verificar si la carpeta existe
if not ruta_muertes.exists():
    print(f"❌ La carpeta {ruta_muertes} no existe")
else:
    # Listar todos los archivos .txt en la carpeta
    archivos = list(ruta_muertes.glob("Defun*.txt"))
    print(f"📁 Archivos encontrados: {[arch.name for arch in archivos]}")
    
    # Filtrar por años 1979-1991
    archivos_filtrados = [arch for arch in archivos if 1979 <= int(arch.stem[-4:]) <= 1991]
    
    dataframes = {}
    
    for archivo in archivos_filtrados:
        try:
            # Extraer año del nombre del archivo
            año = archivo.stem[-4:]
            
            # Leer archivo (prueba diferentes encodings si es necesario)
            df = pd.read_csv(
                archivo,
                sep='\t',        
                encoding='utf-8', 
                low_memory=False
            )
            
            dataframes[f'defun_{año}'] = df
            print(f"✅ {archivo.name}: {len(df)} filas, {len(df.columns)} columnas")
            
        except Exception as e:
            print(f"❌ Error con {archivo.name}: {e}")

📁 Archivos encontrados: ['Defun1979.txt', 'Defun1980.txt', 'Defun1981.txt', 'Defun1982.txt', 'Defun1983.txt', 'Defun1984.txt', 'Defun1985.txt', 'Defun1986.txt', 'Defun1987.txt', 'Defun1988.txt', 'Defun1989.txt', 'Defun1990.txt', 'Defun1991.txt', 'Defun1992.txt', 'Defun1993.txt', 'Defun1994.txt', 'Defun1995.txt', 'Defun1996.txt', 'Defun1997.txt', 'Defun1998.txt', 'Defun1999.txt', 'Defun2000.txt', 'Defun2001.txt', 'Defun2002.txt', 'Defun2003.txt', 'Defun2004.txt', 'Defun2005.txt', 'Defun2006.txt', 'Defun2007.txt', 'Defun2012.txt', 'Defun2013.txt']
✅ Defun1979.txt: 110400 filas, 14 columnas
✅ Defun1980.txt: 125573 filas, 14 columnas
✅ Defun1981.txt: 139505 filas, 14 columnas
✅ Defun1982.txt: 137678 filas, 14 columnas
✅ Defun1983.txt: 140292 filas, 14 columnas
✅ Defun1984.txt: 137189 filas, 14 columnas
✅ Defun1985.txt: 153947 filas, 14 columnas
✅ Defun1986.txt: 146346 filas, 14 columnas
✅ Defun1987.txt: 151957 filas, 14 columnas
✅ Defun1988.txt: 153067 filas, 14 columnas
✅ Defun1989.txt: 1

#### 2.1.1. Consistencia Campos

In [3]:
# Verificar que todas las columnas sean iguales en nombre y orden
nombres_columnas_base = None
columnas_consistentes = True

print("🔍 VERIFICACIÓN DE CONSISTENCIA DE COLUMNAS")
print("=" * 50)

for nombre_df, df in dataframes.items():
    columnas_actuales = df.columns.tolist()
    
    if nombres_columnas_base is None:
        # Establecer la primera como referencia
        nombres_columnas_base = columnas_actuales
        año_base = nombre_df.split('_')[1]
        print(f"📋 DataFrame base (defun_{año_base}): {nombres_columnas_base}")
        print("-" * 50)
    else:
        # Comparar con la base
        if columnas_actuales == nombres_columnas_base:
            print(f"✅ {nombre_df}: Columnas CONSISTENTES")
        else:
            print(f"❌ {nombre_df}: Columnas DIFERENTES")
            print(f"   Esperado: {nombres_columnas_base}")
            print(f"   Encontrado: {columnas_actuales}")
            columnas_consistentes = False

print("=" * 50)
if columnas_consistentes:
    print("🎉 TODAS las columnas son consistentes en nombre y orden")
else:
    print("⚠️  Hay inconsistencias en las columnas")

🔍 VERIFICACIÓN DE CONSISTENCIA DE COLUMNAS
📋 DataFrame base (defun_1979): ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
--------------------------------------------------
❌ defun_1980: Columnas DIFERENTES
   Esperado: ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
   Encontrado: ['cod_dpto', 'cod_munic', 'a_defun', 'ano', 'mes', 'sexo', 'gru_ed1', 'est_civil', 'codptore', 'codmunre', 'sit_defun', 'c_bas1', 'cons_exp', 'cau_homol']
❌ defun_1981: Columnas DIFERENTES
   Esperado: ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
   Encontrado: ['cod_dpto', 'cod_munic', 'a_defun', 'ano', 'mes', 'sexo', 'gru_ed1', 'est_civil', 'codptore', 'codmunre', 'sit_defun', 'c_bas1', '

#### 2.1.2. Estandarización Campos

In [4]:
# Estandarizar todas las columnas a MAYÚSCULAS (como 1979)
print("🔄 ESTANDARIZANDO COLUMNAS A MAYÚSCULAS")
print("=" * 40)

dataframes_estandarizados = {}

for nombre_df, df in dataframes.items():
    # Crear copia del DataFrame
    df_estandar = df.copy()
    
    # Convertir nombres de columnas a MAYÚSCULAS
    df_estandar.columns = [col.upper() for col in df_estandar.columns]
    
    dataframes_estandarizados[nombre_df] = df_estandar
    print(f"✅ {nombre_df}: {list(df.columns)} → {list(df_estandar.columns)}")

# Reemplazar el diccionario original
dataframes = dataframes_estandarizados

🔄 ESTANDARIZANDO COLUMNAS A MAYÚSCULAS
✅ defun_1979: ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL'] → ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
✅ defun_1980: ['cod_dpto', 'cod_munic', 'a_defun', 'ano', 'mes', 'sexo', 'gru_ed1', 'est_civil', 'codptore', 'codmunre', 'sit_defun', 'c_bas1', 'cons_exp', 'cau_homol'] → ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
✅ defun_1981: ['cod_dpto', 'cod_munic', 'a_defun', 'ano', 'mes', 'sexo', 'gru_ed1', 'est_civil', 'codptore', 'codmunre', 'sit_defun', 'c_bas1', 'cons_exp', 'cau_homol'] → ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EX

#### 2.1.3. Verificar Consistencia 2

In [5]:
# Verificar consistencia después de estandarizar
print("\n🔍 VERIFICACIÓN POST-ESTANDARIZACIÓN")
print("=" * 50)

nombres_columnas_base = None
columnas_consistentes = True

for nombre_df, df in dataframes.items():
    columnas_actuales = df.columns.tolist()
    
    if nombres_columnas_base is None:
        nombres_columnas_base = columnas_actuales
        año_base = nombre_df.split('_')[1]
        print(f"📋 DataFrame base (defun_{año_base}): {nombres_columnas_base}")
        print("-" * 50)
    else:
        if columnas_actuales == nombres_columnas_base:
            print(f"✅ {nombre_df}: Columnas CONSISTENTES")
        else:
            print(f"❌ {nombre_df}: Columnas DIFERENTES")
            columnas_consistentes = False

print("=" * 50)
if columnas_consistentes:
    print("🎉 TODAS las columnas ahora son consistentes")
else:
    print("⚠️  Aún hay inconsistencias")


🔍 VERIFICACIÓN POST-ESTANDARIZACIÓN
📋 DataFrame base (defun_1979): ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
--------------------------------------------------
✅ defun_1980: Columnas CONSISTENTES
✅ defun_1981: Columnas CONSISTENTES
✅ defun_1982: Columnas CONSISTENTES
✅ defun_1983: Columnas CONSISTENTES
✅ defun_1984: Columnas CONSISTENTES
✅ defun_1985: Columnas CONSISTENTES
✅ defun_1986: Columnas CONSISTENTES
✅ defun_1987: Columnas CONSISTENTES
✅ defun_1988: Columnas CONSISTENTES
✅ defun_1989: Columnas CONSISTENTES
✅ defun_1990: Columnas CONSISTENTES
✅ defun_1991: Columnas CONSISTENTES
🎉 TODAS las columnas ahora son consistentes


#### 2.1.4. Unificar en un sólo DF defunciones 1979-1991

In [6]:
# Unificar todos los DataFrames en uno solo
print("🔄 UNIFICANDO DATAFRAMES (1979-1991)")
print("=" * 50)

# Verificar que todos los DataFrames están listos
print(f"📊 DataFrames a unificar: {len(dataframes)}")
for nombre, df in dataframes.items():
    print(f"   {nombre}: {df.shape}")

# Unificar todos los DataFrames
defun_1979_1991 = pd.concat(dataframes.values(), ignore_index=True)

print(f"\n✅ UNIFICACIÓN COMPLETADA")
print(f"📈 Tamaño del DataFrame unificado: {defun_1979_1991.shape}")
print(f"   - Filas: {len(defun_1979_1991):,}")
print(f"   - Columnas: {len(defun_1979_1991.columns)}")

🔄 UNIFICANDO DATAFRAMES (1979-1991)
📊 DataFrames a unificar: 13
   defun_1979: (110400, 14)
   defun_1980: (125573, 14)
   defun_1981: (139505, 14)
   defun_1982: (137678, 14)
   defun_1983: (140292, 14)
   defun_1984: (137189, 14)
   defun_1985: (153947, 14)
   defun_1986: (146346, 14)
   defun_1987: (151957, 14)
   defun_1988: (153067, 14)
   defun_1989: (154694, 14)
   defun_1990: (154685, 14)
   defun_1991: (163692, 14)

✅ UNIFICACIÓN COMPLETADA
📈 Tamaño del DataFrame unificado: (1869025, 14)
   - Filas: 1,869,025
   - Columnas: 14


In [7]:
defun_1979_1991.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,SIT_DEFUN,C_BAS1,CONS_EXP,CAU_HOMOL
0,50,1,1,1979,1,2,14,1,50,1,1,6370,2,78
1,50,223,1,1979,1,2,4,1,50,223,2,2762,3,43
2,50,223,1,1979,1,2,24,1,50,223,2,2762,3,43
3,50,683,1,1979,1,1,18,2,50,683,2,8259,3,90
4,50,287,1,1979,1,1,15,1,50,287,2,9660,3,101
